# Top-Level Consistency Checks

This notebook checks the top-level `tiles-SPECPROD.(fits|csv)` and `exposures-SPECPROD.(fits|csv)` files for self-consistency in prepartion for loading these files into a database.

## Imports

In [1]:
import os
import glob
from pytz import utc
import numpy as np
from astropy.io import fits
from astropy.table import Table, Column
from astropy.time import Time
from desiutil.iers import freeze_iers
from desiutil.log import get_logger, DEBUG, INFO
from desispec.io.meta import faflavor2program
from desispec.io.util import checkgzip
import desispec.database.redshift as dsr
from desispec.database.util import targetphotid
from desitarget.targets import decode_targetid
freeze_iers()

INFO:iers.py:82:freeze_iers: Freezing IERS table used by astropy time, coordinates.


## Important Setup

In [2]:
specprod = os.environ['SPECPROD'] = 'fuji'
overwrite = True

In [3]:
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.environ['DESI_ROOT'], 'public', 'edr', 'spectro', 'redux')

## Examine ztile files.

In [12]:
all_targetphotid = list()
ztile_files = glob.glob(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, 'zcatalog', 'ztile-*.fits'))
for f in ztile_files:
    survey = os.path.basename(f).split('-')[1]
    if 'cumulative' in f:
        key = 'LASTNIGHT'
    elif 'perexp' in f:
        key = 'EXPID'
    elif 'pernight' in f:
        key = 'NIGHT'
    else:
        key = 'SPGRPVAL'
    t = Table.read(f, format='fits', hdu='ZCATALOG')
    objid, brickid, release, mock, sky, gaiadr = decode_targetid(t['TARGETID'])
    # print(sky)
    assert (t[key] == t['SPGRPVAL']).all()
    w = (t['TARGETID'] > 0) & (sky == 0)
    all_targetphotid += [targetphotid(x, y, survey) for x, y in zip(t['TARGETID'][w], t['TILEID'][w])]

## Test loading targeting data

In [5]:
release = 'edr'
targetphot_version = 'v1.0'
os.environ['DESI_SPECTRO_REDUX'] = os.path.join(os.environ['DESI_ROOT'], 'public', release, 'spectro', 'redux')
targetphot_potential_file = os.path.join(os.environ['DESI_ROOT'], 'public', release, 'vac', 'lsdr9-photometry', os.environ['SPECPROD'], targetphot_version, 'potential-targets', f"targetphot-potential-{os.environ['SPECPROD']}.fits")
targetphot_potential = Table.read(targetphot_potential_file, hdu=1)

In [6]:
targetphot_file = os.path.join(os.environ['DESI_ROOT'], 'public', release, 'vac', 'lsdr9-photometry', os.environ['SPECPROD'], targetphot_version, 'observed-targets', f"targetphot-{os.environ['SPECPROD']}.fits")
targetphot = Table.read(targetphot_file, hdu=1)

### Check for uniqueness.

In [7]:
targetphot_potential_id = [targetphotid(x, y, z) for x, y, z in zip(targetphot_potential['TARGETID'], targetphot_potential['TILEID'], targetphot_potential['SURVEY'])]

In [8]:
targetphot_id = [targetphotid(x, y, z) for x, y, z in zip(targetphot['TARGETID'], targetphot['TILEID'], targetphot['SURVEY'])]

In [ ]:
for t in targetphot_id:
    assert t in targetphot_potential_id

In [16]:
for a in all_targetphotid:
    assert a in targetphot_id

KeyboardInterrupt: 

### Do ztile files contain extraneous targets?

In [ ]:
ztile = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, 'zcatalog', 'ztile-special-dark-cumulative.fits'), hdu='ZCATALOG')

In [ ]:
w = ztile['TARGETID'] > 0

In [ ]:
np.unique(ztile['TILEID'])

In [ ]:
tid = [targetphotid(x, y, 'special') for x, y in zip(ztile['TARGETID'][w], ztile['TILEID'][w])]

In [13]:
(158457821078535886096914448642 >> 64) & (2**32 - 1)

81101

In [14]:
(158457821078535886096914448642 >> 64) >> 32

2

In [15]:
(158457821078535886096914448642) & (2**64 - 1)

616088991480938754

In [ ]:
ztile[ztile['TARGETID'] == 616088991480938754]

In [ ]:
targetphot[targetphot['TARGETID'] == 616088991480938754]

In [ ]:
targetphot_potential[targetphot_potential['TARGETID'] == 616088991480938754]

In [ ]:
targetphot[targetphot['TILEID'] == 81101]

### Test load targetphot files.

In [ ]:
os.environ['DESI_LOGLEVEL'] = 'DEBUG'
dsr.log = get_logger()
postgresql = dsr.setup_db(schema=specprod+'_target', overwrite=overwrite, hostname='nerscdb03.nersc.gov', verbose=True)

In [ ]:
loader = [{'filepaths': [os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-{specprod}.fits'.format(specprod=os.environ['SPECPROD'])),
                         os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-potential-targets-{specprod}.fits'.format(specprod=os.environ['SPECPROD'])),
                         os.path.join('/global/cscratch1/sd/ioannis/photocatalog', os.environ['SPECPROD'], 'targetphot-missing-{specprod}.fits'.format(specprod=os.environ['SPECPROD']))],
           'tcls': dsr.Target,
           'hdu': 'TARGETPHOT',
           'expand': {'DCHISQ': ('dchisq_psf', 'dchisq_rex', 'dchisq_dev', 'dchisq_exp', 'dchisq_ser',)},
           'q3c': 'ra',
           'chunksize': 100000,
           'maxrows': 0
           },]


In [ ]:
dsr.load_file(**(loader[0]))

## Consistency checks on exposures, frames and tiles

### Load two versions of tiles file

In [ ]:
tiles_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.fits'), hdu='TILE_COMPLETENESS')

In [ ]:
tiles_fits

In [ ]:
tiles_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.csv'), format='ascii.csv')

In [ ]:
tiles_csv

### Are the two tiles files self-consistent?

In [ ]:
for row in range(len(tiles_fits)):
    for col in tiles_fits.colnames:
        try:
            assert tiles_fits[row][col] == tiles_csv[row][col]
        except AssertionError:
            print(tiles_fits[row]['TILEID'], col, tiles_fits[row][col], tiles_csv[row][col])

### Load two versions of exposures file

In [ ]:
exposures_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='EXPOSURES')
frames = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='FRAMES')

In [ ]:
exposures_fits

In [ ]:
exposures_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.csv'), format='ascii.csv')

In [ ]:
exposures_csv

### Are the two exposures files self-consistent?

In [ ]:
for row in range(len(exposures_fits)):
    for col in exposures_fits.colnames:
        try:
            assert exposures_fits[row][col] == exposures_csv[row][col]
        except AssertionError:
            try:
                assert np.around(exposures_fits[row][col].astype(float), 1) == exposures_csv[row][col]
            except AssertionError:
                try:
                    assert np.around(exposures_fits[row][col].astype(float), 2) == exposures_csv[row][col]
                except AssertionError:
                    try:
                        assert np.around(exposures_fits[row][col].astype(float), 3) == exposures_csv[row][col]
                    except AssertionError:
                        print(exposures_fits[row]['TILEID'], col, exposures_fits[row][col], exposures_csv[row][col])

### What programs are present?

In [ ]:
np.unique(faflavor2program(exposures_fits['FAFLAVOR']))

In [ ]:
np.unique(exposures_fits['PROGRAM'])

In [ ]:
np.unique(exposures_fits['GOALTYPE'])

In [ ]:
np.unique(faflavor2program(tiles_fits['FAFLAVOR']))

In [ ]:
np.unique(tiles_fits['PROGRAM'])

In [ ]:
assert (faflavor2program(tiles_fits['FAFLAVOR']) == tiles_fits['PROGRAM']).all

In [ ]:
program = faflavor2program(exposures_fits['FAFLAVOR'])
assert (exposures_fits['PROGRAM'] == program).all()

In [ ]:
for survey in np.unique(exposures_fits['SURVEY']):
    print(f"'{survey}': ", np.unique(program[exposures_fits['SURVEY'] == survey]).tolist(), ',', sep='')

### Compare frames to exposures

In [ ]:
for expid in frames['EXPID']:
    assert expid in exposures_fits['EXPID']

In [ ]:
for k, expid in enumerate(exposures_fits['EXPID']):
    assert (frames['NIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['NIGHT']).all()
    assert (frames['TILEID'][frames['EXPID'] == expid] == exposures_fits[k]['TILEID']).all()
    assert (frames['TILERA'][frames['EXPID'] == expid] == exposures_fits[k]['TILERA']).all()
    assert (frames['TILEDEC'][frames['EXPID'] == expid] == exposures_fits[k]['TILEDEC']).all()
    assert (frames['AIRMASS'][frames['EXPID'] == expid] == exposures_fits[k]['AIRMASS']).all()
    assert (frames['SEEING_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['SEEING_ETC']).all()
    try:
        assert (frames['EFFTIME_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['EFFTIME_ETC']).all()
    except AssertionError:
        print('EFFTIME_ETC', expid, exposures_fits[k]['TILEID'], exposures_fits[k]['EFFTIME_ETC'], frames[column][frames['EFFTIME_ETC'] == expid].tolist())
    # assert (frames['TSNR2_GPBDARK'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBDARK']).all()
    # assert (frames['TSNR2_ELG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_ELG']).all()
    # assert (frames['TSNR2_GPBBRIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBRIGHT']).all()
    # assert (frames['TSNR2_LYA'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LYA']).all()
    # assert (frames['TSNR2_BGS'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_BGS']).all()
    # assert (frames['TSNR2_GPBBACKUP'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBACKUP']).all()
    # assert (frames['TSNR2_QSO'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_QSO']).all()
    # assert (frames['TSNR2_LRG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LRG']).all()
    assert (frames['SURVEY'][frames['EXPID'] == expid] == exposures_fits[k]['SURVEY']).all()
    assert (frames['GOALTYPE'][frames['EXPID'] == expid] == exposures_fits[k]['GOALTYPE']).all()
    assert (frames['FAPRGRM'][frames['EXPID'] == expid] == exposures_fits[k]['FAPRGRM']).all()
    assert (frames['FAFLAVOR'][frames['EXPID'] == expid] == exposures_fits[k]['FAFLAVOR']).all()
    assert (frames['MINTFRAC'][frames['EXPID'] == expid] == exposures_fits[k]['MINTFRAC']).all()
    for column in ('MJD', 'EXPTIME', 'GOALTIME'):
        if column == 'GOALTIME' and (frames[column][frames['EXPID'] == expid] == 0).all():
            print(f"GOALTIME discrepancy for {expid}.")
        else:
            try:
                assert (np.around(frames[column][frames['EXPID'] == expid], 2) == np.around(exposures_fits[k][column], 2)).all()
            except AssertionError:
                try:
                    assert (np.around(frames[column][frames['EXPID'] == expid], 3) == np.around(exposures_fits[k][column], 3)).all()
                except AssertionError:
                    try:
                        assert (np.around(frames[column][frames['EXPID'] == expid], 4) == np.around(exposures_fits[k][column], 4)).all()
                    except AssertionError:
                        pass
                        # print(column, expid, exposures_fits[k][column], frames[column][frames['EXPID'] == expid].tolist())

### Compare tiles to exposures

In [ ]:
for row in tiles_fits:
    w = exposures_fits['TILEID'] == row['TILEID']
    assert len(exposures_fits[w]) == row['NEXP']
    for column in ('SURVEY', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
        try:
            assert (exposures_fits[w][column] == row[column]).all()
        except AssertionError:
            print(row['TILEID'], row[column])
            print(exposures_fits[w][column])
    for column in ('TILERA', 'TILEDEC'):
        try:
            assert (np.around(exposures_fits[w][column], 2) == row[column]).all()
        except AssertionError:
            try:
                assert (np.around(exposures_fits[w][column], 3) == row[column]).all()
            except AssertionError:
                assert (np.around(exposures_fits[w][column], 4) == row[column]).all()
    for column in ('EXPTIME', 'EFFTIME_SPEC', 'LRG_EFFTIME_DARK', 'ELG_EFFTIME_DARK', 'BGS_EFFTIME_BRIGHT', 'LYA_EFFTIME_DARK'):
        try:
            assert np.allclose(np.around(exposures_fits[w][column].sum(), 1), row[column])
        except AssertionError:
            print(row['TILEID'], row[column])
            print(column, np.around(exposures_fits[w][column].sum(), 1))
    for column in ('EFFTIME_ETC', 'EFFTIME_GFA'):
        if (exposures_fits[w][column] == 0).any():
            assert row[column] == 0
        else:
            try:
                assert np.allclose(np.around(exposures_fits[w][column].sum(), 1), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.around(exposures_fits[w][column].sum(), 1))
    for column in ('GOALTIME', 'MINTFRAC'):
        try:
            assert np.allclose(np.around(exposures_fits[w][column], 1), row[column])
        except AssertionError:
            try:
                assert np.allclose(np.around(exposures_fits[w][column], 2), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.round(exposures_fits[w][column], decimals=1))
    
    assert exposures_fits[w]['NIGHT'].max() == row['LASTNIGHT']